# Applications

## Tagging - POS

**Parts of speed tagging** - Gán nhãn loại từ:

- *POS Tagging* là quá trình gán nhãn (label) loại từ cho mỗi từ trong một câu, ví dụ như danh từ (N), động từ (V), tính từ (ADJ), trạng từ (ADV), giới từ (PREP), liên từ (CONJ), v.v.
- *Mục tiêu*: Từ một chuỗi các từ, dự đoán lớp từ tương ứng sao cho phản ánh đúng ngữ pháp và ngữ nghĩa trong ngôn ngữ đó.

**Ý nghĩa**

- Xác định loại từ là bước cơ bản và cần thiết trong nhiều ứng dụng NLP phức tạp hơn, ví dụ:
    - *Parsing*: Xây dựng cây phân tích cú pháp (syntactic parsing).
    - *Dịch máy*: Hỗ trợ xác định cấu trúc ngữ pháp, lựa chọn từ tương ứng chính xác.
    - *Trích xuất thông tin*: Hỗ trợ nhận diện và phân loại từ, cụm từ đặc biệt.
- POS Tagging giúp máy tính hiểu được chức năng ngữ pháp của từ trong câu, từ đó xử lý ngôn ngữ hiệu quả hơn.

**Phương pháp**

- *Thống kê cổ điển (HMM, CRF)*: Các mô hình ẩn Markov (HMM) và Mô hình trường ngẫu nhiên có điều kiện (CRF) dựa vào xác suất chuyển tiếp giữa các nhãn và xác suất quan sát từ để dự đoán nhãn POS.
- *Học sâu (Deep Learning)*: Các mô hình mạng nơ-ron (RNN, LSTM, BiLSTM, Transformers) kết hợp với embeddings (như Word2Vec, GloVe, hay contextual embeddings như BERT) cũng có thể được huấn luyện để gán nhãn POS một cách chính xác hơn, nhất là với những ngôn ngữ giàu ngữ cảnh như tiếng Việt.

In [2]:
import spacy
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import matplotlib.pyplot as plt

data = pd.read_csv(
    r"contents\theory\aiml_algorithms\dl_nlp\data\94 - bbc-news.csv"
)
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Preprocessing
titles = data[["title"]].copy()

# lower case
titles["preprocessed"] = titles["title"].str.lower()

# stop word removal
en_stopwords = stopwords.words("english")
titles["preprocessed"] = titles["preprocessed"].apply(
    lambda x: " ".join(
        [word for word in x.split() if word not in en_stopwords]
    )
)

# punctation removal
titles["preprocessed"] = titles["preprocessed"].apply(
    lambda x: re.sub(r"[^\w\s]", "", x)
)

# word tokenization
titles["tokens_raw"] = titles["title"].apply(word_tokenize)
titles["tokens_preprocessed"] = titles["preprocessed"].apply(word_tokenize)

# lemmatizing
lemmatizer = WordNetLemmatizer()
titles["tokens_lemmatized"] = titles["tokens_preprocessed"].apply(
    lambda x: [lemmatizer.lemmatize(word) for word in x]
)

# create lists for just our tokens
tokens_raw_list = sum(
    titles["tokens_raw"], []
)  # unpack our lists into a single list
tokens_clean_list = sum(titles["tokens_lemmatized"], [])

In [5]:
# create a spacy doc from our raw text - better for pos tagging
spacy_doc = nlp(" ".join(tokens_raw_list))

In [ ]:
# POS Tagging
pos = [{"token": token.text, "pos_tag": token.pos_} for token in spacy_doc]
pd.DataFrame(pos).head()

,token,pos_tag
0,Can,AUX
1,I,PRON
2,refuse,VERB
3,to,PART
4,work,VERB


## Tagging - NER

**Named Entity Recognition** – Nhận diện thực thể có tên

- *NER* là quá trình tự động phát hiện (dò tìm) và phân loại các thực thể “có tên” trong văn bản, ví dụ như tên người, tên địa điểm, tổ chức, thời gian, sự kiện, v.v.
- *Mục tiêu*: Xác định đâu là thực thể đặc biệt trong câu và chúng thuộc loại thực thể nào.
```text
Ví dụ câu tiếng Việt: “Ông Nguyễn Văn A làm việc tại Công ty Bkav ở Hà Nội.”
“Nguyễn Văn A” → Người (Person)
“Công ty Bkav” → Tổ chức (Organization)
“Hà Nội” → Địa điểm (Location)
```

**Ý nghĩa** 

NER là một bước quan trọng trong trích xuất thông tin:

- *Tìm kiếm và tổng hợp thông tin*: Cho phép hệ thống hiểu được đối tượng nào đang được đề cập đến (ví dụ, tìm tất cả các tin tức có nhắc đến một người/tổ chức cụ thể).
- *Hỏi đáp tự động (Question Answering)*: Giúp xác định câu trả lời chính xác khi câu hỏi hướng đến các thực thể (Ví dụ: “Ai là CEO của Google?”).
- *Phân tích ý kiến (Sentiment Analysis)*: Xem ý kiến của người dùng nhắm vào thực thể nào (nhãn hàng, địa điểm, cá nhân).

**Phương pháp**

- *Rule-based*: Sử dụng luật ngôn ngữ hoặc biểu thức chính quy (regex), kết hợp danh sách các thực thể đã biết. Cách này mang tính đặc thù, thiếu linh hoạt.
- *Thống kê/học máy cổ điển*: CRF, MaxEnt (Maximum Entropy), SVM… sử dụng đặc trưng (feature) về từ, ngữ cảnh, chữ hoa/thường, tiền tố/hậu tố, v.v.
- *Học sâu (Deep Learning)*:
    - Mô hình mạng nơ-ron nhiều tầng (BiLSTM + CRF) hoặc Transformers (BERT, XLM-R, PhoBERT…) huấn luyện đặc trưng tự động, thay vì thủ công.
    - Kết quả NER hiện nay cải thiện rất nhiều nhờ áp dụng mô hình Transformer, đặc biệt khi có dữ liệu gắn nhãn đủ lớn.

In [ ]:
ner = [
    {"token": token.text, "ner_tag": token.label_} for token in spacy_doc.ents
]
df = pd.DataFrame(ner).head()
df

,token,ner_tag
0,Liz Truss,PERSON
1,UK,GPE
2,Rationing,PRODUCT
3,superyachts,CARDINAL
4,Russian,NORP


In [ ]:
from spacy import displacy

displacy.render(spacy_doc[:150], style="ent", jupyter=True)

In [19]:
type(spacy_doc)

spacy.tokens.doc.Doc

## Sentiment Analysis

**Mục tiêu**: Nhằm gán nhãn (label) “tích cực”, “tiêu cực” hoặc “trung tính” cho một đoạn văn, câu, hoặc thậm chí ở mức độ chi tiết hơn (từng ý hoặc từ).

**Ứng dụng thực tế**:
- Phân tích phản hồi (review) sản phẩm, khách sạn, nhà hàng.
- Khai thác ý kiến trên mạng xã hội (Twitter, Facebook, v.v.).
- Xây dựng hệ thống hỗ trợ chăm sóc khách hàng tự động.
- Theo dõi và quản trị danh tiếng thương hiệu.

**Các phương pháp phổ biến trong Sentiment Analysis**

*1. Phương pháp dựa trên từ điển/ngữ nghĩa (Rule-based / Lexicon-based)*

- Từ điển cảm xúc (sentiment lexicon): Là bộ danh sách gồm các từ ngữ chứa hàm ý cảm xúc (từ tích cực như “tuyệt vời”, “đẹp”, “xuất sắc”, từ tiêu cực như “tệ”, “xấu”, “kinh khủng”,...). Mỗi từ thường gắn với một trọng số (score) phản ánh mức độ tích cực hoặc tiêu cực.
- Luật và quy tắc (rule): Áp dụng các quy tắc (ví dụ: nếu một câu có nhiều từ ngữ tiêu cực hơn thì khả năng cao câu đó mang nghĩa tiêu cực).

Ưu điểm:
- Dễ triển khai khi có sẵn từ điển cảm xúc.
- Giải thích được kết quả (biết từ nào quyết định cảm xúc).

Nhược điểm:
- Độ bao phủ (coverage) giới hạn nếu từ điển không đầy đủ.
- Khó nắm bắt bối cảnh (context) phức tạp, ví dụ mệnh đề phủ định hay các câu mỉa mai, châm biếm (sarcasm).
- Khó bảo trì và mở rộng từ điển theo ngôn ngữ, ngữ cảnh mới.

*2. Phương pháp Deep Learning Transformer*

- Transformer-based Models (BERT, RoBERTa, GPT, XLM-R, PhoBERT với tiếng Việt, v.v.)
- Transformer đã trở thành kiến trúc chuẩn cho nhiều bài toán NLP, bao gồm Sentiment Analysis.
- Các mô hình Pre-trained như BERT có khả năng biểu diễn ngôn ngữ tự nhiên rất tốt nhờ quá trình huấn luyện trên dữ liệu cực lớn.
- Khi fine-tuning cho bài toán phân tích cảm xúc, mô hình có thể đạt kết quả cao vượt trội.

Ưu điểm:
- Khả năng nắm bắt ngữ cảnh và mối quan hệ giữa các từ tốt hơn so với phương pháp cổ điển.
- Tính khái quát cao, mở rộng được cho các ngôn ngữ, miền dữ liệu khác nhau.

Nhược điểm:
- Đòi hỏi dữ liệu huấn luyện lớn, tài nguyên phần cứng (GPU/TPU) mạnh.
- Khó giải thích kết quả hơn so với phương pháp rule-based hay machine learning truyền thống.

**Quy trình triển khai một hệ thống Sentiment Analysis**
1. Thu thập dữ liệu: Lấy dữ liệu đánh giá, phản hồi, bình luận, v.v.
2. Tiền xử lý & Chuẩn hóa: Làm sạch, tách câu/từ, loại bỏ nhiễu, xử lý ký tự đặc biệt.
3. Chọn chiến lược tiếp cận:
- Rule-based / Lexicon-based: Cần có bộ từ điển cảm xúc phong phú, các quy tắc xử lý ngôn ngữ phù hợp.
- Machine Learning: Cần gán nhãn dữ liệu, trích xuất đặc trưng và huấn luyện.
- Deep Learning: Áp dụng mô hình embedding, RNN/CNN/Transformer, fine-tuning.
4. Huấn luyện và đánh giá:
- Chia dữ liệu thành các bộ train/validation/test.
- Đánh giá mô hình qua các chỉ số: Accuracy, Precision, Recall, F1-score.
5. Triển khai và giám sát:
- Đưa mô hình vào môi trường chạy thật.
- Thu thập phản hồi, đánh giá và tiếp tục cải thiện.

In [1]:
data = [
    {
        "text": "Sản phẩm này quá tuyệt vời, rất đáng đồng tiền.",
        "sentiment": "positive",
    },
    {
        "text": "Thật sự thất vọng, chất lượng kém hơn mong đợi.",
        "sentiment": "negative",
    },
    {
        "text": "Chưa biết xài sao, mới mua thôi, chắc ổn.",
        "sentiment": "neutral",
    },
    {
        "text": "Quá chán, mất thời gian, không đáng tiền chút nào.",
        "sentiment": "negative",
    },
    {
        "text": "Khá tốt, giá hợp lý, nhưng giao hàng hơi chậm.",
        "sentiment": "neutral",
    },
    {
        "text": "Thiết kế đẹp, màu sắc đúng ý, sẽ mua lại nếu cần.",
        "sentiment": "positive",
    },
    {
        "text": "Tuy sản phẩm không đẹp,  nhưng hài lòng chất lượng.",
        "sentiment": "positive",
    },
]

label2id = {"negative": 0, "neutral": 1, "positive": 2}
id2label = {v: k for k, v in label2id.items()}


### Pre-trained model

In [8]:
from transformers import pipeline

model_path = '5CD-AI/Vietnamese-Sentiment-visobert'
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model=model_path,
    tokenizer=model_path
)

for item in data:
    result = sentiment_pipeline(item["text"])[0]
    item["predicted_sentiment"] = result["label"]
    item["score"] = result["score"]

import pandas as pd
pd.DataFrame(data)

config.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/390M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/471k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

Device set to use cpu


,text,sentiment,predicted_sentiment,score
0,"Sản phẩm này quá tuyệt vời, rất đáng đồng tiền.",positive,POS,0.998884
1,"Thật sự thất vọng, chất lượng kém hơn mong đợi.",negative,NEG,0.998847
2,"Chưa biết xài sao, mới mua thôi, chắc ổn.",neutral,POS,0.659783
3,"Quá chán, mất thời gian, không đáng tiền chút ...",negative,NEG,0.999171
4,"Khá tốt, giá hợp lý, nhưng giao hàng hơi chậm.",neutral,POS,0.726378
5,"Thiết kế đẹp, màu sắc đúng ý, sẽ mua lại nếu cần.",positive,POS,0.993708
6,"Tuy sản phẩm không đẹp, nhưng hài lòng chất l...",positive,NEU,0.678071


### Custom model

train test split

In [2]:
from sklearn.model_selection import train_test_split

# Chuyển data sang list
data_list = list(data)

# Bước 1: tách train (khoảng 60%) và tạm_thời (40%)
train_data, temp_data = train_test_split(data_list, test_size=0.4, random_state=42, shuffle=True)

# Bước 2: từ temp_data tách ra validation (50%) và test (50%)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42, shuffle=True)

print("Train size:", len(train_data))
print("Val size:", len(val_data))
print("Test size:", len(test_data))

Train size: 4
Val size: 1
Test size: 2


Tạo Dataset cho Hugging Face

In [3]:
import torch
from torch.utils.data import Dataset

class SentimentDataset(Dataset):
    def __init__(self, data, tokenizer, label2id, max_len=64):
        """
        data: list of dict, mỗi dict có {"text": str, "sentiment": str}
        tokenizer: tokenizer từ Hugging Face (AutoTokenizer, v.v.)
        label2id: dict ánh xạ từ sentiment (str) -> id (int)
        max_len: chiều dài tối đa khi tokenize
        """
        self.data = data
        self.tokenizer = tokenizer
        self.label2id = label2id
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        text = sample["text"]
        sentiment = sample["sentiment"]
        label_id = self.label2id[sentiment]

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt",
        )

        # Mặc định, tokenizer trả về:
        #   encoding["input_ids"], encoding["attention_mask"], (có thể có token_type_ids)
        # Ta squeeze(0) để loại bỏ chiều batch (vì mỗi mẫu chỉ có 1)
        input_ids = encoding["input_ids"].squeeze(0)
        attention_mask = encoding["attention_mask"].squeeze(0)

        # Tạo dict kết quả
        item = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": torch.tensor(label_id, dtype=torch.long)
        }

        return item




In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_path = "5CD-AI/Vietnamese-Sentiment-visobert"
tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    num_labels=3,
    id2label=id2label,
    label2id=label2id
)

train_dataset = SentimentDataset(train_data, tokenizer, label2id, max_len=64)
val_dataset   = SentimentDataset(val_data,   tokenizer, label2id, max_len=64)
test_dataset  = SentimentDataset(test_data,  tokenizer, label2id, max_len=64)

Metrics for evaluation

In [11]:
import numpy as np

def evaluate_function(eval_pred):
    logits, labels = eval_pred  # logits, labels đều là numpy.ndarray
    # Dùng np.argmax thay vì torch.argmax
    predictions = np.argmax(logits, axis=-1)

    # Tính metric (VD: accuracy, f1...)
    # Ví dụ dùng scikit-learn hoặc huggingface metric
    # Ở đây minh hoạ load_metric("accuracy") và load_metric("f1")
    # (Bạn có thể thay bằng metric tuỳ ý)

    import evaluate

    accuracy_metric = evaluate.load("accuracy")
    f1_metric = evaluate.load("f1")

    acc = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')

    return {
        "accuracy": acc["accuracy"],
        "f1": f1["f1"]
    }


Training

In [12]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./output/visobert-finetuned",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    logging_dir="./output/logs",
    logging_steps=1,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # dùng để validate
    processing_class=tokenizer,       # Trainer sẽ tự động batch & pad
    compute_metrics=evaluate_function
)

trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.001200,5.580416,0.000000,0.000000
2,0.000300,8.432305,0.000000,0.000000
3,0.000500,8.990547,0.000000,0.000000


C:\Users\datkt\Desktop\Working\notebooks\coding\learning\.venv\Lib\site-packages\transformers\configuration_utils.py:397: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 256}
  warnings.warn(
C:\Users\datkt\Desktop\Working\notebooks\coding\learning\.venv\Lib\site-packages\transformers\configuration_utils.py:397: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This 

TrainOutput(global_step=6, training_loss=0.0007009292385191657, metrics={'train_runtime': 35.6154, 'train_samples_per_second': 0.337, 'train_steps_per_second': 0.168, 'total_flos': 394670126592.0, 'train_loss': 0.0007009292385191657, 'epoch': 3.0})

Evaluation

In [14]:
test_metrics = trainer.evaluate(test_dataset)
print("Test metrics:", test_metrics)

Test metrics: {'eval_loss': 3.8895528316497803, 'eval_accuracy': 0.5, 'eval_f1': 0.5, 'eval_runtime': 2.8813, 'eval_samples_per_second': 0.694, 'eval_steps_per_second': 0.347, 'epoch': 3.0}


In [18]:
test_text = "Hàng giao nhanh, chất lượng vừa ý."
inputs = tokenizer(test_text, return_tensors="pt", truncation=True, padding=True)

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    pred_id = logits.argmax(dim=-1).item()

print("Text:", test_text)
print("Predicted sentiment:", id2label[pred_id])


Text: Hàng giao nhanh, chất lượng vừa ý.
Predicted sentiment: neutral
